In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform

print ("initialization completed")

initialization completed


In [2]:
class Bird_Dataset_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, transform=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
        label = int(img_dir_label[1])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [3]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([448, 448]),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),   
        transforms.Normalize([0.480, 0.511, 0.534], [0.221, 0.215, 0.246])
    ]),
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([448, 448]),
        transforms.CenterCrop(448),
        transforms.ToTensor(),
        transforms.Normalize([0.480, 0.511, 0.534], [0.221, 0.215, 0.246])
    ]),
}

root_dir = '/home/xinsir/Pytorch/air_data/'
train_file = '/home/xinsir/Pytorch/air_data/train_list.txt'
test_file = '/home/xinsir/Pytorch/air_data/test_list.txt'
image_datasets = {'train': Bird_Dataset_Process(train_file, root_dir, data_transforms['train']),
                  'test': Bird_Dataset_Process(test_file, root_dir, data_transforms['test'])}

print(type(image_datasets['train']))
print(image_datasets['train'][0]['image'].shape)
print(len(image_datasets['train']))

train_dataset = image_datasets['train']
test_dataset = image_datasets['test']

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}


dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

print(dataset_sizes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

<class '__main__.Bird_Dataset_Process'>
torch.Size([3, 448, 448])
6667
{'test': 3333, 'train': 6667}


In [4]:
# train_matrix = torch.Tensor(6667, 3, 448, 448)
# print(train_matrix.shape)
# for i in range(len(train_dataset)):
#     print(i)
#     train_matrix[i] = train_dataset[i]['image']
# R = np.mean(train_matrix[:, 0, :, :].numpy().flat)
# G = np.mean(train_matrix[:, 1, :, :].numpy().flat)
# B = np.mean(train_matrix[:, 2, :, :].numpy().flat)
# R_std = np.std(train_matrix[:, 0, :, :].numpy().flat)
# G_std = np.std(train_matrix[:, 1, :, :].numpy().flat)
# B_std = np.std(train_matrix[:, 2, :, :].numpy().flat)
# print(R, G, B, R_std, G_std, B_std)

In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
                
        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            start_time = time.time()        
            if epoch % 5 == 0 and phase == 'train':
                for param_group in optimizer.param_groups:
                    lr = param_group['lr']
                    print("***********************")
                    print("learning rate = %f" % lr)
            
            # Iterate over data.
            for i_batch, sample_batched in enumerate(dataloaders[phase]):
                inputs = sample_batched['image']
                labels = sample_batched['label']
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                if i_batch % 50 == 0 and phase == 'train':
                    print("Iteration %d, loss = %f" % (i_batch, loss))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'test':
                scheduler.step(epoch_loss)
                
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            cost_time = (time.time() - start_time)/60.0
            print('{} 1 epoch time: {:.2f}min'.format(
                phase, cost_time))
            
            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [6]:
vgg_16 = models.vgg16(pretrained=True)


vgg_16.classifier = nn.Sequential(*list(vgg_16.classifier.children())[:1])
vgg_16.features[30] = nn.AvgPool2d(kernel_size=28, stride=1, padding=0)
vgg_16.features.add_module('31', nn.Dropout(0.8)) 
vgg_16.classifier[0] = nn.Linear(in_features=512, out_features=200, bias=True)

# vgg_16 = torch.load('/home/xinsir/Pytorch/aircraft_vgg16')
vgg_16 = vgg_16.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
optimizer_ft = optim.SGD(vgg_16.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='min', factor=0.5, patience=10)

model_best = train_model(vgg_16, criterion, optimizer_ft,
                         exp_lr_scheduler, num_epochs=1)

Epoch 0/0
----------
test Loss: 0.6541 Acc: 0.8668
test 1 epoch time: 0.64min

Training complete in 0m 38s
Best val Acc: 0.866787


In [7]:
torch.save(model_best, '/home/xinsir/Pytorch/aircraft_vgg16_0.9_86.7%')